# Audio classification model inference

* Model - pretrained fastai2 xresnet18 using fastai2 audio library

**fastai2_audio**

The additional requirements of the fastai2_audio package will be dealt with below, using a clone of the following repo:

https://github.com/rbracco/fastai2_audio

The demo was run and tested by deploying an SageMaker Notebook instance as per the instructions outlined [here] (https://forums.fast.ai/t/platform-amazon-sagemaker-aws/66020).

Note - the above link is only accessible as part of the ongoing fastai course for the time being.


## Note re dependencies

These are set up using the LifeCycle Configuration for the notebook files. The original fastai2 LifeCycle Configuration provided by Matt McClean https://forums.fast.ai/t/fastai2-sagemaker/66444/6 has been modified to also have the installed of the fastai2 audio github repo:

`!pip install git+https://github.com/mikful/fastai2_audio.git`


As such, the below Installations of fastai2 and fastai2 audio are not necessary if using this LifeCycle Configuration.

However, the installation of the libsndfile is required for to avoid and OSError (this will need to be placed within the LifeCycle Config for the setup at some point.

`!conda install -c conda-forge libsndfile --yes`

## Install fastai2

**Note: not required if using fastai2 + fastai2 audio LifeCyCle Config**

In [ ]:
#In SageMaker we need to run this as a  shell commands i.e. with '!' infront of 'pip'
#!pip install fastai2

## Install the fastai2_audio library

We need to install the fastai2_audio library to the local kernel/environment for the analysis

**Note: not required if using fastai2 + fastai2 audio LifeCyCle Config**

In [4]:
#In Colab we need to run this as a shell command i.e. with '!' infront of 'pip'

#!pip install git+https://github.com/mikful/fastai2_audio.git

In [2]:
# Solving an OSError problem with Librosa SoundFile dependency (libsndfile)
# SageMaker/GCP Only

!conda install -c conda-forge libsndfile --yes

Solving environment: done

## Package Plan ##

  environment location: /home/ec2-user/SageMaker/.env/fastai2

  added / updated specs:
    - libsndfile


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gettext-0.19.8.1           |       h5e8e0c9_1         3.5 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

  gettext            conda-forge/linux-64::gettext-0.19.8.1-h5e8e0c9_1
  libflac            conda-forge/linux-64::libflac-1.3.3-he1b5a44_0
  libogg             conda-forge/linux-64::libogg-1.3.2-h516909a_1002
  libsndfile         conda-forge/linux-64::libsndfile-1.0.28-he1b5a44_1000
  libvorbis          conda-forge/linux-64::libvorbis-1.3.6-he1b5a44_2
  python_abi         conda-forge/linux-64::python_abi-3.6-1_cp36m

The following packages wi

## Download Dataset

In this case needed to create the labels for the learner

The dataset will be downloaded directly from Kaggle using the steps outlined within the following Kaggle Forum post:

https://www.kaggle.com/general/74235

## Load Pretrained Model (from Colab) and Perform Inference

In [1]:
from fastai2.vision.all import *
from fastai2_audio.core import *
from fastai2_audio.augment import *

/home/ec2-user/SageMaker/.env/fastai2/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
DBMelSpec = SpectrogramTransformer(mel=True, to_db=True)

In [3]:
cfg = AudioConfig.BasicMelSpectrogram()
aud2spec = AudioToSpec.from_cfg(cfg)
aud2spec.settings

{'mel': True,
 'to_db': True,
 'sample_rate': 16000,
 'n_fft': 400,
 'win_length': 400,
 'hop_length': 200,
 'f_min': 0.0,
 'f_max': None,
 'pad': 0,
 'n_mels': 128,
 'window_fn': <function _VariableFunctions.hann_window>,
 'wkwargs': None,
 'stype': 'power',
 'top_db': None}

In [4]:
# Now let's change the settings to see the impact
aud2spec = DBMelSpec(sample_rate= 16000, f_max=None, f_min=20, n_mels=128, n_fft=1024, hop_length=128, top_db=90)
aud2spec.settings

{'mel': True,
 'to_db': True,
 'sample_rate': 16000,
 'n_fft': 1024,
 'win_length': 1024,
 'hop_length': 128,
 'f_min': 20,
 'f_max': None,
 'pad': 0,
 'n_mels': 128,
 'window_fn': <function _VariableFunctions.hann_window>,
 'wkwargs': None,
 'stype': 'power',
 'top_db': 90}

In [5]:
item_tfms = [RemoveSilence(), CropSignal(3000, pad_mode='Repeat'), 
             DownmixMono(), aud2spec, MaskTime(num_masks=1, size=100), MaskFreq(num_masks=1, size=10)]

## Option 1 - Read files from S3 bucket into SageMaker for testing - Not working

https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-dg.pdf
pg. 38

**Note**
> Amazon SageMaker needs permission to access S3 buckets. You grant permission with an IAM role, which you create in the next step when you create an Amazon SageMaker notebook instance. This IAM role automatically gets permissions to access any bucket that has sagemaker in the name. It gets these permissions through the AmazonSageMakerFullAccess policy, which Amazon SageMaker attaches to the role. If you add a policy to the role that grants the SageMaker service principal S3FullAccess permission, the name of the bucket does not need to contain sagemaker.

In [7]:
# access S3 file directly
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
bucket='audio-app-mf-ct/uploads'
data_key = 'audio-2020-05-06T20-41-06-257Z.wav'
data_location = 's3://{}/{}'.format(bucket, data_key)
#data_location = 's3://{}'.format(bucket)
print(data_location)

arn:aws:iam::445129834695:role/audio-app-mf-ct-Fastai2SagemakerNotebookfastaiv4No-CFY4HZCTHRO
s3://audio-app-mf-ct/uploads/audio-2020-05-06T20-41-06-257Z.wav


## Option 2  - Copy files from S3 bucket to SageMaker for testing - Working

In [73]:
# copy files

!aws s3 cp s3://audio-app-mf-ct/uploads/audio-2020-05-06T20-23-23-844Z.wav test-audio/

download: s3://audio-app-mf-ct/uploads/audio-2020-05-06T20-23-23-844Z.wav to test-audio/audio-2020-05-06T20-23-23-844Z.wav


In [7]:
# check files 
print(audio_extensions)
fnames = get_files("../audio-app-mf-ct/test-audio", extensions=audio_extensions)
fnames

('.aif', '.aifc', '.aiff', '.au', '.m3u', '.mp2', '.mp3', '.ra', '.ram', '.snd', '.wav', '.726', '.ac3', '.amr', '.awb', '.aal', '.atx', '.at3', '.aa3', '.omg', '.dls', '.evc', '.evb', '.evw', '.lbc', '.l16', '.mxmf', '.mpga', '.mp1', '.oga', '.ogg', '.spx', '.sid', '.psid', '.qcp', '.smv', '.koz', '.eol', '.mlp', '.dts', '.dtshd', '.plj', '.lvp', '.pya', '.vbk', '.ecelp4800', '.ecelp7470', '.ecelp9600', '.smp3', '.smp', '.s1m', '.mid', '.midi', '.kar', '.mod', '.ult', '.uni', '.m15', '.mtm', '.669', '.med', '.wax', '.wma', '.rm', '.s3m', '.stm')


(#2) [Path('../audio-app-mf-ct/test-audio/audio-2020-05-06T20-41-06-257Z.wav'),Path('../audio-app-mf-ct/test-audio/audio-2020-05-06T20-23-23-844Z.wav')]

In [67]:
import IPython.display as ipd
ipd.Audio(fnames[0], rate=44100) # load a WAV file

## Option 3 - Stream files from bucket directly

In [8]:
import boto3 import io s3 = boto3.client('s3') 

bucket='audio-app-mf-ct/uploads'
data_key = 'audio-2020-05-06T20-41-06-257Z.wav'

obj = s3.get_object(Bucket=bucket, Key=data_key) 
image = mpimg.imread(io.BytesIO(obj['Body'].read()), 'jp2')

with open(stream_file, 'rb') as audio_file:
    content = audio_file.read(BYTES_PER_SECOND)

SyntaxError: invalid syntax (<ipython-input-8-93f96da28aa4>, line 1)

## Define DataLoaders and test data

From https://dev.fast.ai/tutorial.pets#Adding-a-test-dataloader-for-inference :

In [18]:
# Define test file path in SageMaker
from pathlib import Path 
path = Path("../audio-app-mf-ct/test-audio")
tst_files = get_audio_files(path)
print(len(tst_files))

# define dataloaders transform
dblock = DataBlock(blocks=(AudioBlock, CategoryBlock),  
                 get_items=get_audio_files, 
                 item_tfms = item_tfms)

#dsets = dblock.datasets(path)
dls = dblock.dataloaders(path, bs=32) # bs= batch_size
dls.train.vocab

2


(#2) [Path('../audio-app-mf-ct/test-audio/audio-2020-05-06T20-23-23-844Z.wav'),Path('../audio-app-mf-ct/test-audio/audio-2020-05-06T20-41-06-257Z.wav')]

In [31]:
### Load pretrained 1-channel xresnet18 with multi-accuracy

# Custom cnn model created from pretrained xresnet18 (smaller model for inference speed)
# 1 input channel and 80 output nodes
# torch.nn.BCEWithLogitsLoss() = Binary Cross Entropy Loss from pytorch
# accuracy_multi for multi label

model = create_cnn_model(xresnet18, n_in=1, n_out=80, pretrained=True)

learn = Learner(dls, model, BCEWithLogitsLossFlat(), metrics=accuracy_multi) # pass custom model to Learner

learn = learn.load('xresnet50-stage-2-model-finetuned')

In [34]:
# define test file and test dataloader
dl = learn.dls.test_dl(tst_files)
   
# # predict using tta
# preds = learn.tta(dl=dl)


file = Path('../audio-app-mf-ct/test-audio/audio-2020-05-06T20-41-06-257Z.wav')
preds = learn.predict(file)

print(preds)

('(#0) []', tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False]), tensor([1.0617e-04, 5.4842e-03, 1.8752e-03, 3.9582e-07, 1.2573e-04, 7.2953e-07,
        1.6282e-02, 6.7340e-07, 1.9695e-06, 8.5559e-07, 5.1405e-05, 1.8603e-03,
        9.3917e-05, 1.5708e-06, 4.7525e-07, 1.6853e-04, 4.1530e-05, 1.8104e-05,
        9.5804e-03, 3.9586e-06, 4.5782e-06, 7.3847e-07, 9.3113e-07, 2.5287e-05,
        8.9076e-05, 5.4539e-06, 2.6587e-06

In [7]:
inference_folder = Path('../audio-app-mf-ct/test-audio')

#exported model predictions - steps

#get audio files to run
audios = get_audio_files(inference_folder);audios

#get model name
name = 'xresnet50-stage-2-model-finetuned'

#load model with file/path
modelex = '../audio-app-mf-ct/models/xresnet50-stage-2-model-finetuned.pth';modelex

# Create a learner to load exported model into
model = create_cnn_model(xresnet18, n_in=1, n_out=80, pretrained=True)
learn = Learner(dls, model, BCEWithLogitsLossFlat(), metrics=accuracy_multi) # pass custom model to Learner

#load exported model
# Use load_learner(path/file) for exported models and 
# learn.load(name) for regular models saved during training.
learn = learn.load('xresnet50-stage-2-model-finetuned')
print("labels:", learn.dls.vocab)
learn.dls.vocab = 
print("labels:", learn.dls.vocab)

#pass in images to create test batch
dl = learn.dls.test_dl(audios)

#dl = test_dl(learn.dls, audios)
_, __, preds = learn.get_preds(dl=dl, with_decoded=True)


# #get preds for batch
# print("Predicting...")
# preds, targs = learn.tta(dl=dl)
# #pred_tensor, ignored, preds = learn.get_preds(dl=dl, with_decoded=True)

#outputs of above
print(f'Prediction Tensors: {preds}')
#tensor([[0.8150, 0.0348, 0.0220, 0.0258, 0.1023]])

# #category index
# print(f'Predictions: {preds}')
# #tensor([0])
# print(len(preds[0]))


#index into vocab to turn int index into label 
print("labels:", learn.dls.vocab[0])
#"my_label

#print(learn.dls.vocab.o2i)
#output full dictionary of category index and label
#{'label1': 0, 'label2': 1, 'label3': 2, 'invalid': 3, 'negative': 4}


# results = learn.predict(audios[0])
# #output of results
# #'category_name', tensor(0), tensor([0.8150, 0.0348, 0.0220, 0.0258, 0.1023]))


# #loop to spit out formatted results from get_preds
# for index,item in enumerate(pred_tensor):
#     prediction = learn.dls.categorize.decode(np.argmax(item)).upper()
#     confidence = max(item)
#     percent = float(confidence)
#     print(f"{prediction}{percent*100:.2f}% confidence. Audio File = {learn.dl.items[index].name}")

# #get file name(s)
# learn.dl.items[0].name

# #show tested image(s)
# learn.dl.show_batch()

NameError: name 'dls' is not defined

## Export the model and upload to S3

Now that we have trained our model we will export it using the learner method `export()` and upload the exported model to S3.

In [ ]:
learn.export()

Now let's create a tarfile for our model.

In [ ]:
import tarfile
with tarfile.open(path/'model.tar.gz', 'w:gz') as f:
    f.add(path/'export.pkl', arcname='model.pkl')

In [ ]:
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()

In [ ]:
prefix = 'audio-app-mf-ct'

Now we will upload the model to the default S3 bucket for sagemaker.

In [ ]:
model_location = sess.upload_data(str(path/'model.tar.gz'), key_prefix=prefix)
model_location

## Script for model inference

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the `model_fn()` is called to determine how to load your trained model. The `model_fn()` along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* `model_fn(model_dir)` - loads your model.
* `input_fn(serialized_input_data, content_type)` - deserializes predictions to predict_fn.
* `output_fn(prediction_output, accept)` - serializes predictions from predict_fn.
* `predict_fn(input_data, model)` - calls a model on data deserialized in input_fn.

Here is the full code in a file `serve.py` showing implementations of the 4 key functions:

In [ ]:
!pygmentize scripts/serve.py

## Deploy locally to test

Before deploying to Amazon SageMaker we want to verify that the endpoint is working properly. The Amazon SageMaker Python SDK allows us to deploy locally to the Notebook instance using Docker. We will create the model then specify the parameter `instance_type` to be `local` telling the SDK to deploy locally.

In [ ]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(model_data=model_location,
                     role=role,
                     framework_version='1.4.0',
                     entry_point='serve.py', 
                     source_dir='scripts')

Now that we have created the model we will deploy locally to test. It may take a while to run the first time as we need to download a Docker image to our notebook instance.

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='local')

Now we can test out our endpoint. We will download a cat images from the internet and save locally.

In [ ]:
! [ -d tmp ] || mkdir tmp
! wget -q -O tmp/british-shorthair.jpg https://cdn1-www.cattime.com/assets/uploads/2011/12/file_2744_british-shorthair-460x290-460x290.jpg

In [ ]:
img = Image.open('tmp/british-shorthair.jpg')
img

Now we can call our local endpoint to ensure it is working and provides us the correct result.

In [ ]:
from sagemaker.predictor import json_serializer, json_deserializer

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

response = predictor.predict( { "url": "https://cdn1-www.cattime.com/assets/uploads/2011/12/file_2744_british-shorthair-460x290-460x290.jpg" })

print(response)

Once you are happy that the endpoint is working suceessully you can shut it down.

In [ ]:
predictor.delete_endpoint()

## Deploy to SageMaker

Once we have verified that the script is working successfully on our locally deployed endpoint we can deploy our model to Amazon SageMaker so that it can be used in a production application. The code is almost exactly the same as deploying locally except that when we call `model.deploy()` we will change the instance type to an Amazon SageMaker valid instance type (e.g. `ml.m5.xlarge`).

In [ ]:
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(model_data=model_location,
                     role=role,
                     framework_version='1.4.0',
                     entry_point='serve.py', 
                     source_dir='scripts')

Now let's deploy our SageMaker endpoint. It will take a few min to provision.

In [ ]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

In [ ]:
img = Image.open('tmp/british-shorthair.jpg')
img

Now let's test our remote endpoint running on SageMaker hosting services.

In [ ]:
from sagemaker.predictor import json_serializer, json_deserializer

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

response = predictor.predict( { "url": "https://cdn1-www.cattime.com/assets/uploads/2011/12/file_2744_british-shorthair-460x290-460x290.jpg" })

print(response)

## Optional: delete endpoint

If you do not want to keep the endpoint up and running then remember to delete it to avoid incurring further costs.

In [ ]:
predictor.delete_endpoint()